# **Load Dataset**

In [2]:
import pandas as pd

df = pd.read_csv('transaction_data.csv')

# set Transaction ID column as index
df.set_index('Transaction ID', inplace=True)
df

,A,C,D,E,I,K,M,N,O,U,Y
Transaction ID,,,,,,,,,,,
1000,0,0,0,1,0,1,1,1,1,0,1
2000,0,0,1,1,0,1,0,0,1,0,1
4000,1,0,0,1,0,1,1,0,0,0,0
5000,0,1,0,0,0,1,1,0,0,1,1
6000,0,1,0,1,1,1,0,0,2,0,0
7000,0,0,1,0,0,0,1,2,2,0,0


# **Menampilkan jumlah transaksi dari masing masing barang**

In [3]:
# lihat jumlah transaksi dari masing masing kolom yang nilainya lebih dari 0
df_sum = df[df > 0].count()
# tampilkan hasilnya dalam dataframe dengan kolom 'barang' dan 'jumlah'
df_sum = pd.DataFrame({
    'Barang': df_sum.index,
    'Jumlah': df_sum.values
})
# tampilkan dataframe
df_sum

,Barang,Jumlah
0,A,1
1,C,2
2,D,2
3,E,4
4,I,1
5,K,5
6,M,4
7,N,2
8,O,4
9,U,1


In [4]:
# filter data untuk jumlah barang yang memiliki transaksi lebih dari atau sama dengan 2
df_filtered = df_sum[df_sum['Jumlah'] >= 2]
# tampilkan DataFrame yang sudah difilter
df_filtered

,Barang,Jumlah
1,C,2
2,D,2
3,E,4
5,K,5
6,M,4
7,N,2
8,O,4
10,Y,3


# **Membuat kombinasi 2 set item dari barang yang ada di tabel sebelumnya**

In [5]:
# buat pasangan untuk setiap barang
from itertools import combinations
# buat pasangan dari kolom Barang
combinations_list = list(combinations(df_filtered['Barang'], 2))
# buat DataFrame dari pasangan
df_combinations = pd.DataFrame(combinations_list, columns=['Barang 1', 'Barang 2'])
# tampilkan DataFrame pasangan
df_combinations

,Barang 1,Barang 2
0,C,D
1,C,E
2,C,K
3,C,M
4,C,N
5,C,O
6,C,Y
7,D,E
8,D,K
9,D,M


In [6]:
# hitung jumlah transaksi untuk setiap pasangan
df_combinations['Jumlah Transaksi'] = df_combinations.apply(
    lambda row: len(df[(df[row['Barang 1']] > 0) & (df[row['Barang 2']] > 0)]), axis=1
)
# tampilkan DataFrame dengan jumlah transaksi
df_combinations = df_combinations[df_combinations['Jumlah Transaksi'] > 0].copy()
df_combinations = df_combinations.sort_values(by='Jumlah Transaksi', ascending=False)
# tampilkan DataFrame akhir
df_combinations.reset_index(drop=True, inplace=True)
df_combinations

,Barang 1,Barang 2,Jumlah Transaksi
0,E,K,4
1,E,O,3
2,K,O,3
3,K,M,3
4,K,Y,3
5,M,N,2
6,E,M,2
7,E,Y,2
8,C,K,2
9,D,O,2


In [7]:
# filter data untuk kolom 'Jumlah Transaksi' pada df_combinations yang lebih besar dari atau sama dengan 2
df_combinations_filtered = df_combinations[df_combinations['Jumlah Transaksi'] >= 2]
# tampilkan DataFrame yang sudah difilter
df_combinations_filtered.reset_index(drop=True, inplace=True)
df_combinations_filtered

,Barang 1,Barang 2,Jumlah Transaksi
0,E,K,4
1,E,O,3
2,K,O,3
3,K,M,3
4,K,Y,3
5,M,N,2
6,E,M,2
7,E,Y,2
8,C,K,2
9,D,O,2


# **Menampilkan kombinasi 3 barang dari tabel sebelumnya**

In [8]:
# Ambil pasangan dari df_combinations_filtered
pair_list = df_combinations_filtered[['Barang 1', 'Barang 2']].values.tolist()

# Gabungkan pasangan yang punya awalan sama
triplet_dict = {}
for pair1 in pair_list:
    for pair2 in pair_list:
        # Cek awalan sama dan ada satu huruf yang berbeda
        if pair1 != pair2 and pair1[0][:1] == pair2[0][:1]:
            # Gabungkan unique items dari kedua pasangan
            triplet = tuple(sorted(set(pair1 + pair2)))
            if len(triplet) == 3:
                triplet_dict[triplet] = None  # gunakan dict untuk menghindari duplikat

# Hitung jumlah transaksi untuk setiap triplet
triplet_data = []
for triplet in triplet_dict:
    count = len(df[(df[triplet[0]] > 0) & (df[triplet[1]] > 0) & (df[triplet[2]] > 0)])
    triplet_data.append({'Triplet': ''.join([t[-1] for t in triplet]),  # misal: OKE, KEY
                        'Barang 1': triplet[0], 'Barang 2': triplet[1], 'Barang 3': triplet[2],
                        'Jumlah Transaksi': count})

df_triplet = pd.DataFrame(triplet_data)
df_triplet = df_triplet[df_triplet['Jumlah Transaksi'] > 0].sort_values(by='Jumlah Transaksi', ascending=False)
df_triplet.reset_index(drop=True, inplace=True)
df_triplet[['Triplet', 'Jumlah Transaksi']]

,Triplet,Jumlah Transaksi
0,EKO,3
1,EKM,2
2,EKY,2
3,EOY,2
4,KMY,2
5,KOY,2
6,MNO,2
7,EMO,1
8,KMO,1
9,MNY,1


In [9]:
# filter jumlah transaksi yang lebih besar dari atau sama dengan 2
df_triplet_filtered = df_triplet[df_triplet['Jumlah Transaksi'] >= 2]
# tampilkan DataFrame triplet yang sudah difilter
df_triplet_filtered.reset_index(drop=True, inplace=True)
df_triplet_filtered[['Triplet', 'Jumlah Transaksi']]

,Triplet,Jumlah Transaksi
0,EKO,3
1,EKM,2
2,EKY,2
3,EOY,2
4,KMY,2
5,KOY,2
6,MNO,2


# **Menampilkan kombinasi subset dari tabel triplet**

In [10]:
from itertools import combinations

# Ambil triplet dari df_triplet_filtered
subsets = []
for _, row in df_triplet_filtered.iterrows():
    triplet = [row['Barang 1'], row['Barang 2'], row['Barang 3']]
    # Subset ukuran 1
    for item in triplet:
        subsets.append({'Triplet': ''.join([b[-1] for b in triplet]), 'Subset': '{' + item[-1] + '}'})
    # Subset ukuran 2
    for pair in combinations(triplet, 2):
        subsets.append({'Triplet': ''.join([b[-1] for b in triplet]), 'Subset': '{' + pair[0][-1] + ', ' + pair[1][-1] + '}'})

df_subsets = pd.DataFrame(subsets)
df_subsets

,Triplet,Subset
0,EKO,{E}
1,EKO,{K}
2,EKO,{O}
3,EKO,"{E, K}"
4,EKO,"{E, O}"
5,EKO,"{K, O}"
6,EKM,{E}
7,EKM,{K}
8,EKM,{M}
9,EKM,"{E, K}"


# **Membuat Dataframe kombinasi anteseden dan konsekuen, support, serta confidence nya**

In [11]:
from itertools import combinations

confidence_list = []

for _, row in df_triplet_filtered.iterrows():
    triplet = [row['Barang 1'], row['Barang 2'], row['Barang 3']]
    triplet_support = row['Jumlah Transaksi']
    triplet_name = ''.join([b[-1] for b in triplet])
    
    # Semua subset non-kosong dan bukan triplet penuh
    for r in [1, 2]:
        for antecedent in combinations(triplet, r):
            consequent = tuple(sorted(set(triplet) - set(antecedent)))
            # Hitung support antecedent
            mask = (df[list(antecedent)] > 0).all(axis=1)
            antecedent_support = mask.sum()
            if antecedent_support > 0:
                confidence = triplet_support / antecedent_support * 100
                confidence_list.append({
                    'Triplet': triplet_name,
                    'Aturan': f"{set([a[-1] for a in antecedent])} => {set([c[-1] for c in consequent])}",
                    'Support Antecedent': antecedent_support,
                    'Support Triplet': triplet_support,
                    'Confidence (%)': round(confidence, 2)
                })

df_confidence = pd.DataFrame(confidence_list)
df_confidence

,Triplet,Aturan,Support Antecedent,Support Triplet,Confidence (%)
0,EKO,"{'E'} => {'K', 'O'}",4,3,75.00
1,EKO,"{'K'} => {'O', 'E'}",5,3,60.00
2,EKO,"{'O'} => {'K', 'E'}",4,3,75.00
3,EKO,"{'K', 'E'} => {'O'}",4,3,75.00
4,EKO,"{'O', 'E'} => {'K'}",3,3,100.00
5,EKO,"{'K', 'O'} => {'E'}",3,3,100.00
6,EKM,"{'E'} => {'K', 'M'}",4,2,50.00
7,EKM,"{'K'} => {'M', 'E'}",5,2,40.00
8,EKM,"{'M'} => {'K', 'E'}",4,2,50.00
9,EKM,"{'K', 'E'} => {'M'}",4,2,50.00


In [12]:
# filter df_confidence yang memiliki confidence lebih dari atau sama dengan 80%
df_confidence_filtered = df_confidence[df_confidence['Confidence (%)'] >= 80]
df_confidence_filtered.reset_index(drop=True, inplace=True)
df_confidence_filtered

,Triplet,Aturan,Support Antecedent,Support Triplet,Confidence (%)
0,EKO,"{'O', 'E'} => {'K'}",3,3,100.0
1,EKO,"{'K', 'O'} => {'E'}",3,3,100.0
2,EKM,"{'M', 'E'} => {'K'}",2,2,100.0
3,EKY,"{'E', 'Y'} => {'K'}",2,2,100.0
4,EOY,"{'E', 'Y'} => {'O'}",2,2,100.0
5,EOY,"{'O', 'Y'} => {'E'}",2,2,100.0
6,KMY,"{'M', 'Y'} => {'K'}",2,2,100.0
7,KOY,"{'O', 'Y'} => {'K'}",2,2,100.0
8,MNO,"{'N'} => {'M', 'O'}",2,2,100.0
9,MNO,"{'N', 'M'} => {'O'}",2,2,100.0


## Interpretasi Hasil

Berdasarkan analisis asosiasi pada data transaksi, berikut adalah beberapa temuan utama:

- **Frekuensi Barang**: Barang dengan jumlah transaksi terbanyak adalah K (5 kali), diikuti oleh E, M, dan O (masing-masing 4 kali).
- **Kombinasi Barang**: Kombinasi dua barang yang paling sering muncul bersama adalah E dan K (4 transaksi), diikuti oleh kombinasi lain seperti E-O, K-O, K-M, dan K-Y.
- **Kombinasi Tiga Barang**: Kombinasi tiga barang yang paling sering muncul bersama adalah E, K, dan O (3 transaksi). Kombinasi lain seperti E-K-M, E-K-Y, E-O-Y, K-M-Y, K-O-Y, dan M-N-O muncul sebanyak 2 transaksi.
- **Aturan Asosiasi**: Aturan asosiasi dengan confidence 100% menunjukkan hubungan yang sangat kuat, misalnya:
    - Jika pelanggan membeli N, maka mereka juga membeli O dan M.
    - Jika membeli kombinasi {E, O}, maka pasti membeli K.
    - Jika membeli kombinasi {O, K}, maka pasti membeli E.
- **Confidence**: Confidence 100% berarti setiap kali antecedent (misal {N}) terjadi, consequent (misal {O, M}) juga selalu terjadi dalam data.

Hasil ini dapat digunakan untuk strategi pemasaran, seperti penempatan barang secara berdampingan atau promosi bundling pada barang-barang yang sering dibeli bersamaan.